# 15. More About Type Hints

> I learned a painful lesson that for small programs, dynamic typing is great. For large programs you need a more disciplined approach. And it helps if the language gives you that discipline rather than telling you "Well, you can do whatever you want".
>
>> Guido Van Rossum

## Overloaded Signatures

Python functions may accept different combinations of arguments. The `@typing.overload` decorator allows annotating those different combiations. This is particularly important when the return type of the function depends on the type of two or more parameters.

As mentioned before, the two leading undersecores in a parameter like `__iterable` are a PEP 484 convention for positional-only arguments that is enforced by Mypy. It meas you can call `sum(my_list)`, but not `sum(__iterable = my_list)`.

In [1]:
import functools
import operator
from collections.abc import Iterable
from typing import overload, Union, TypeVar

T = TypeVar('T')
S = TypeVar('S')

@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...
@overload
def sum(it: Iterable[T], /, star: S) -> Union[T, S]: ...
def sum(it, /, start=0):
    return functools.reduce(operator.add, it, start)

## Max Overload

It is difficult to add type hints to functions that leverage the powerful dynamic features of Python.

In [2]:
print(f"{max(1, 2, -3, key=abs)=}")

max(1, 2, -3, key=abs)=-3


In [3]:
print(f"{max( ['Go', 'Python', 'Rust'] )=}")

max( ['Go', 'Python', 'Rust'] )='Rust'


In [4]:
print(f"{max( [1, 2, -3], default=0)=}")

max( [1, 2, -3], default=0)=2


In [5]:
print(f"{max( [], default=None)=}")

max( [], default=None)=None


In [6]:
print(f"{max( [1, 2, -3], key=abs, default=None)=}")

max( [1, 2, -3], key=abs, default=None)=-3


In [7]:
print(f"{max( [], key=abs, default=None)=}")

max( [], key=abs, default=None)=None


### Takeaways from Overloading max

Type hints allow Mypy to flag a call like `max([None, None])` with this error message:

```text
mymax_demo.py:109: error: Value of type variable "_LT" of "max" cannot be "None"
```

On the other hand, having to write so many lines to support the type checker may discourage people from writing convenient and flexible functions like `max`.

## TypedDict

Python dictionaries are sometimes used as records, with the keys used as field names and field values of different types.

```python
from typing import TypedDict

class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int
```

At first glance, `typing.TypedDict` may seem like a data class builder, similar to `typing.NamedTuple`.

The syntactic similarity is misleading. `TypedDict` is very different. It exists only for the benefit of type checkers, and has no runtime effect.

In [8]:
from typing import TypedDict

class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int

pp = BookDict(title='Programming Pearls',
              authors='Jon Bentley',
              isbn='0201657880',
              pagecount=256,)
pp

{'title': 'Programming Pearls',
 'authors': 'Jon Bentley',
 'isbn': '0201657880',
 'pagecount': 256}

In [9]:
type(pp)

dict

In [10]:
import sys

try:
    print(f"{pp.title=}")
except Exception as e: print(f"{e=}", file=sys.stderr)

e=AttributeError("'dict' object has no attribute 'title'")


In [11]:
pp['title']

'Programming Pearls'

In [12]:
BookDict.__annotations__

{'isbn': str, 'title': str, 'authors': list[str], 'pagecount': int}

In [13]:
from typing import TYPE_CHECKING

def demo() -> None:
    book = BookDict(
        isbn = '0134757599',
        title = 'Refactoring, 2e',
        authors = ['Martin Fowler', 'Kent Beck', ],
        pagecount = 478,
    )
    authors = book['authors']
    if TYPE_CHECKING:
        reveal_type(authors)
    authors = 'Bob'
    book['weight'] = 4.2
    del book['title']

demo()

## Type Casting

No type system is perfect, and neither are the static type checkers, the type hints in the _typeshed_ project, or the type hints in the third-party packages that have them.

The `typing.cast()` special function provides one way to handle type checking malfunctions or incorrect type hints in code we can’t fix. 

At runtime, `typing.cast` does absolutely nothing. This is its implementation:

```python

def cast(typ, val):
    """Cast a value to a type.
    This returns the value unchanged. To the type checker this
    signals that the return value has the designated type, but at 
    runtime we intentionally don't check anything (we want this 
    to be as fast as possible).
    """
    return val
```

PEP 484 requires type checkers to “blindly believe” the type stated in the `cast`.

## Reading Type Hints at Runtime

At import time, Python reads the type hints in functions, classes, and modules, and stores them in attributes named `__annotations__`. For instance, consider the `clip` function:

```python
def clip(text: str, max_len: int = 80) -> str:
```

The type hints are stored as a dict in the `__annotations__` attribute of the function:

```python
>>> from clip_annot import clip
>>> clip.__annotations__
{'text': <class 'str'>, 'max_len': <class 'int'>, 'return': <class 'str'>}
```

The `'return'` key maps to the return type hint after the `->` symbol.

### Problems with Annotations at Runtime

The increased use of type hints raised two problems:
+ Importing modules uses more CPU and memory when many type hints are used.
+ Referring to types not yet defined requires using strings instead of actual types.

Both issues are relevant. The first is because of what we just saw: annotations are evaluated by the interpreter at import time and stored in the `__annotations__` attribute. Let’s focus now on the second issue.

## Implementing a Generic Class

Before, we defined the `Tombola` ABC: an interface for classes that work like a bingo cage. Also before, The `LottoBlower` class is a concrete implementation.

```python
from generic_lotto import LottoBlower

machine = LottoBlower[int]( range(1, 11) )

first = machine.pick()
remain = machine.inspect()
```

```python
from collections.abc import Iterable
from typing import TypeVar, Generic

from tombola import Tombola

T = TypeVar('T')

class LottoBlower(Tombola, Generic[T]):

    def __init__(self, items: Iterable[T]) -> None:
        self._balls = list[T](items)

    def load(self, items: Iterable[T]) -> None:
        self._balls.extend(items)

    def pick(self) -> T:
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)
    
    def loaded(self) -> bool:
        return bool(self._balls)
    
    def inspect(self) -> tuple[T, ...]:
        return tuple(self._balls)
````

### Basic Jargon for Generic Types

Here are a few definitions that I found useful when studying generics:

_Generic type_

  A type declared with one or more type variables.

  Examples: `LottoBlower[T]`, `abc.Mapping[KT, VT]`

_Formal type parameter_

  The type variables that appear in a generic type declaration.

  Example: `KT` and `VT` in the previous example `abc.Mapping[KT, VT]`

_Parameterized type_

  A type declared with actual type parameters.

  Examples: `LottoBlower[int]`, `abc.Mapping[str, float]`

_Actual type parameter_

  The actual types given as parameters when a parameterized type is declared.

  Example: the `int` in `LottoBlower[int]`

### Variance Review

Variance is a subtle property.

#### Invariant Types

A generic type `L` is invariant when there is no supertype or subtype relationship between two parameterized types, regardless of the relationship that may exist between the actual parameters. In other words, if `L` is invariant, then `L[A]` is not a supertype or a subtype of `L[B]`. They are inconsistent in both ways.

As mentioned, Python’s mutable collections are invariant by default. The `list` type is a good example: `list[int]` is not _consistent-with_ `list[float]` and vice versa.

In general, if a formal type parameter appears in type hints of method arguments, and the same parameter appears in method return types, that parameter must be invariant to ensure type safety when updating and reading from the collection.

#### Covariant Types

Consider two types `A` and `B`, where `B` is _consistent-with_ `A`, and neither of them is `Any`. Some authors use the `<:` and `:>` symbols to denote type relationships like this:

`A :> B`
    A is a _supertype-of_ or the same as B

`B <: A`
    B is a _subtype-of_ or the same as A

Given `A :> B`, a generic type `C` is covariant when `C[A] :> C[B]`.

Note the direction of the `:>` symbol is the same in both cases where `A` is to the left of `B`. Covariant generic types follow the subtype relationship of the actual type parameters.

Immutable containers can be covariant. For example, this is how the `typing.Frozen Set` class is documented as a covariant with a type variable using the conventional name `T_co`:

```python
    class FrozenSet(frozenset, AbstractSet[T_co]):
```

Applying the `:>` notation to parametrized types, we have:

               float :> int                 
    frozenset[float] :> frozenset[int]      

Iterators are another example of covariant generics: they are not read-only collections like a `frozenset`, but they only produce output. Any code expecting an `abc.Iterator[float]` yielding floats can safely use an `abc.Iterator[int]` yielding integers. `Callable` types are covariant on the return type for a similar reason.

#### Contravariant Types

Given `A :> B`, a generic type `K` is contravariant if `K[A] <: K[B]`.

Contravariant generic types reverse the subtype relationship of the actual type parameters.

The `TrashCan` class exemplifies this:

              Refuse :> Biodegradable               
    TrashCan[Refuse] <: TrashCan[Biodegradable]     

A contravariant container is usually a write-only data structure, also known as a “sink.” There are no examples of such collections in the standard library, but there are a few types with contravariant type parameters.

#### Variance Rules of Thumb

Finally, here are a few rules of thumb to reason about when thinking through variance:

+ If a formal type parameter defines a type for data that comes out of the object, it can be covariant.
+ If a formal type parameter defines a type for data that goes into the object after its initial construction, it can be contravariant.
+ If a formal type parameter defines a type for data that comes out of the object and the same parameter defines a type for data that goes into the object, it must be invariant.
+ To err on the safe side, make formal type parameters invariant.